In [45]:
#20/5/24 Testing Quantization and other size reduction methods for an autoencoder model in keras
#One hot models tend to get absurdly large and thus this can help qol for moving and testing these models




import pandas as pd
import numpy as np
#Data import
targetdata = pd.read_csv('/Users/james/Documents/Honours/Python/Target_Selection/Drug_Targets/DrugTargets.csv')

#Creating test/train splits
from sklearn.model_selection import train_test_split

trainset, testset = train_test_split(targetdata, test_size=0.2, random_state=81)
#Converts Toxicity Values to a list
ytrain = trainset['Toxicity_Values'].to_list
ytest = testset['Toxicity_Values'].to_list
#Converts encoded drug target values to an array
xtrain = trainset.iloc[:, 2:].values
xtest = testset.iloc[:, 2:].values
xtest2 = testset.iloc[:, 2:]

In [38]:
xtest2

,"('CHEMBL4768',)","('CHEMBL226',)","('CHEMBL3571',)","('CHEMBL3706',)","('CHEMBL5979',)","('CHEMBL2035',)","('CHEMBL2652',)","('CHEMBL3714079',)","('CHEMBL1968',)","('CHEMBL329',)",...,"('CHEMBL252',)","('CHEMBL1628481',)","('CHEMBL1792',)","('CHEMBL208',)","('CHEMBL2888',)","('CHEMBL3991',)","('CHEMBL228',)","('CHEMBL3776',)","('CHEMBL3108639',)","('CHEMBL325',)"
9498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
452,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8297,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12383,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11672,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9691,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
#The tested methods have proven ineffective in model size reduction

In [30]:
#Pruning Implementation
#Does not reduce model size
import tensorflow_model_optimization as tfmot
lowmagprune = tfmot.sparsity.keras.prune_low_magnitude

import tensorflow as tf
from tensorflow import keras

# Determines layer size and input/output
# Here it turns the 334 one hot encoded features to 42
latent_space = 42
keras.utils.set_random_seed(81)
tf.config.experimental.enable_op_determinism()

# Encoder Creation
enc_in = keras.Input(shape=(334, 1))
x = keras.layers.Flatten()(enc_in)

# Additional encoder layers
x = keras.layers.Dense(160, activation="relu")(x)
x = keras.layers.Dense(81, activation="relu")(x)

enc_out = keras.layers.Dense(latent_space, activation="relu")(x)
encoder = keras.Model(enc_in, enc_out, name='encoder')

# Decoder Creation
decoder_in = keras.layers.Dense(latent_space, activation='relu')(enc_out)

# Additional decoder layers
x = keras.layers.Dense(81, activation='relu')(decoder_in)
x = keras.layers.Dense(160, activation='relu')(x)

x = keras.layers.Dense(334, activation='relu')(x)
decoder_out = keras.layers.Reshape((334,1))(x)

autoencoder = keras.Model(enc_in, decoder_out, name='autoencoder')

lowmagprune = tfmot.sparsity.keras.prune_low_magnitude
end_step = np.ceil(len(xtrain) / 3).astype(np.int32) * 4
params = {'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
          initial_sparsity=0.5,
          final_sparsity=0.8,
          begin_step=0,
          end_step=end_step)}

autoencoder = lowmagprune(autoencoder, **params)

# Optimiser
optimise = tf.keras.optimizers.Adam()

# Model Creation
#autoencoder = keras.Model(enc_in, decoder_out, name='autoencoder')

autoencoder.compile(optimise, loss='binary_crossentropy')
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 334, 1)]          0         
                                                                 
 prune_low_magnitude_flatten  (None, 334)              1         
 _6 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_dense_4  (None, 160)              107042    
 2 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_dense_4  (None, 81)               26003     
 3 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_dense_4  (None, 42)               6848      
 4 (PruneLowMagnitude)                                 

In [34]:
#Runs and creates the autoencoder and outputs parameters to a specs file
epochs = 4
batchsize = 3
latent_space = 42
import tempfile
logdir = tempfile.mkdtemp()
for epoch in range(epochs):
    
    callbacks = [
        tfmot.sparsity.keras.UpdatePruningStep(),
        tfmot.sparsity.keras.PruningSummaries(log_dir=logdir)
    ]
    
    logs = autoencoder.fit(
    xtrain,
    xtrain,
    epochs=1,
    batch_size=batchsize, validation_split=0.2,
    callbacks = callbacks)
    
    autoencoder.save(str(f"prunmodels/AE-{epoch+1}.keras"))

   1/2826 [..............................] - ETA: 1:07 - loss: 0.0902WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0010s vs `on_train_batch_end` time: 0.0036s). Check your callbacks.


2024-05-20 20:35:50.461460: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [8476,334]
	 [[{{node Placeholder/_11}}]]
2024-05-20 20:35:50.461629: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [8476,334]
	 [[{{node Placeholder/_11}}]]


2826/2826 [==============================] - 3s 952us/step - loss: 0.3831 - val_loss: 0.3919


2024-05-20 20:35:52.999031: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [2119,334]
	 [[{{node Placeholder/_11}}]]
2024-05-20 20:35:52.999162: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [2119,334]
	 [[{{node Placeholder/_11}}]]


   1/2826 [..............................] - ETA: 1:01 - loss: 0.0983WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0009s vs `on_train_batch_end` time: 0.0035s). Check your callbacks.


...

2024-05-20 20:35:58.375711: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [2119,334]
	 [[{{node Placeholder/_11}}]]
2024-05-20 20:35:58.375856: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [2119,334]
	 [[{{node Placeholder/_11}}]]


   1/2826 [..............................] - ETA: 55s - loss: 0.0807WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0009s vs `on_train_batch_end` time: 0.0032s). Check your callbacks.


2024-05-20 20:35:58.620615: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [8476,334]
	 [[{{node Placeholder/_10}}]]
2024-05-20 20:35:58.620748: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [8476,334]
	 [[{{node Placeholder/_11}}]]


2826/2826 [==============================] - 3s 918us/step - loss: 0.3812 - val_loss: 0.3684


2024-05-20 20:36:01.047487: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [2119,334]
	 [[{{node Placeholder/_11}}]]
2024-05-20 20:36:01.047613: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [2119,334]
	 [[{{node Placeholder/_11}}]]


In [19]:
import tensorflow as tf
from tensorflow import keras
import os

autoencoder = keras.models.load_model('models/AE-7.keras')

In [3]:
#Not in use, not compatible with files >2gb


#Testing Post Training Quantization


converter = tf.lite.TFLiteConverter.from_keras_model(autoencoder)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quant_model = converter.convert()

with open('test.tflite', 'wb') as f:
    f.write(quant_model)

2024-05-20 13:01:42.260268: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]
2024-05-20 13:01:42.345598: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: /var/folders/cq/5cl8ry195dj0219k6rt6csbm0000gn/T/tmpejypa455/assets


2024-05-20 13:01:42.842813: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-05-20 13:01:42.842833: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-05-20 13:01:42.843705: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/cq/5cl8ry195dj0219k6rt6csbm0000gn/T/tmpejypa455
2024-05-20 13:01:42.844709: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-05-20 13:01:42.844715: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/cq/5cl8ry195dj0219k6rt6csbm0000gn/T/tmpejypa455
2024-05-20 13:01:42.847663: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2024-05-20 13:01:42.848585: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-05-20 13:01:42.850766: W tensorflow/tsl/platform/profile_u

In [17]:
#Working with a tensorflowlite model (the one created from quantization)

interpreter = tf.lite.Interpreter('test.tflite')
interpreter.allocate_tensors()

inputs = interpreter.get_input_details()
outputs = interpreter.get_output_details()

shape = inputs[0]['shape']
data = np.array(xtest2)
interpreter.set_tensor(inputs[0]['index'], data)

interpreter.invoke()

test = interpreter.get_tensor(outputs[0]['index'])
print(test)
#signature = interpreter.get_signature_runner()
#out = signature(x=tf.constant([1.0], shape=(1,10), dtype=tf.float32))

INFO: Applying 1 TensorFlow Lite delegate(s) lazily.
INFO: Failed to apply the default TensorFlow Lite delegate indexed at 0 because of incompatibility between runtime and delegate. Ignoring the error, and continuing anyway.


ValueError: Cannot set tensor: Got value of type INT64 but expected type FLOAT32 for input 0, name: serving_default_input_1:0 

In [18]:
xtest2

,"('CHEMBL4768',)","('CHEMBL226',)","('CHEMBL3571',)","('CHEMBL3706',)","('CHEMBL5979',)","('CHEMBL2035',)","('CHEMBL2652',)","('CHEMBL3714079',)","('CHEMBL1968',)","('CHEMBL329',)",...,"('CHEMBL252',)","('CHEMBL1628481',)","('CHEMBL1792',)","('CHEMBL208',)","('CHEMBL2888',)","('CHEMBL3991',)","('CHEMBL228',)","('CHEMBL3776',)","('CHEMBL3108639',)","('CHEMBL325',)"
9498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
452,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8297,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12383,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11672,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9691,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
#Gathers the accuracy, fp and tp rate of the autoencoder on a test set
ae_out = autoencoder.predict([ xtest.reshape(-1, 334, 1) ])
test = ae_out.reshape(2649,334)

ids = 0
acc = []
fplist = []
tplist = []

for things in test:
    guesses = []
    testset = []

    loops = 0
    for items in xtest[ids]:
        if items != 0:
            testset.append(loops)
        loops = loops + 1

    loops = 0
    for items in test[ids]:
        if items > 0.5:
            guesses.append(loops)
        loops = loops + 1

    correct = 0
    loops = 0
    tp = 0
    fp = 0
    for items in guesses:
        if items in testset:
            correct = correct + 1
            tp = tp + 1
        else:
            fp = fp + 1
        loops = loops + 1

    accuracy = correct / loops
    tprate = tp / len(guesses)
    fprate = fp / len(guesses)

    acc.append(accuracy)
    tplist.append(tprate)
    fplist.append(fprate)

    ids = ids + 1

totalacc = 0
totaltp = 0
totalfp = 0

loops = 0
for items in acc:
    totalacc = totalacc + items
    loops = loops + 1
netacc = totalacc/loops

loops = 0
for items in tplist:
    totaltp = totaltp + items
    loops = loops + 1
nettp = totaltp/loops

loops = 0
for items in fplist:
    totalfp = totalfp + items
    loops = loops + 1
netfp = totalfp/loops

netacc = totalacc/loops
nettp = totaltp/loops
netfp = totalfp/loops

#Calculates fn and tn rate

ids = 0
tnlist = []
fnlist = []

for things in test:
    guesses = []
    testset = []

    loops = 0
    for items in xtest[ids]:
        if items != 1:
            testset.append(loops)
        loops = loops + 1

    loops = 0
    for items in test[ids]:
        if items < 0.5:
            guesses.append(loops)
        loops = loops + 1

    correct = 0
    loops = 0
    tn = 0
    fn = 0
    for items in guesses:
        if items in testset:
            correct = correct + 1
            tn = tn + 1
        else:
            fn = fn + 1
        loops = loops + 1

    tnrate = tn / len(guesses)
    fnrate = fn / len(guesses)

    tnlist.append(tprate)
    fnlist.append(fprate)

    ids = ids + 1

totaltn = 0
totalfn = 0

loops = 0
for items in tnlist:
    totaltn = totaltn + items
    loops = loops + 1
nettn = totaltn/loops

loops = 0
for items in fnlist:
    totalfn = totalfn + items
    loops = loops + 1
netfn = totalfn/loops

nettn = totaltn/loops
netfn = totalfn/loops



83/83 [==============================] - 0s 450us/step


2024-05-20 20:36:02.069867: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [2649,334,1]
	 [[{{node Placeholder/_10}}]]
2024-05-20 20:36:02.070040: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [2649,334,1]
	 [[{{node Placeholder/_10}}]]


In [36]:
print('fn rate =', netfn)
print('tn rate =', nettn)
print('net accuracy =', netacc)
print('fp rate =', netfp)
print('tp rate =', nettp)

fn rate = 0.25
tn rate = 0.75
net accuracy = 0.8552390552058609
fp rate = 0.14476094479413892
tp rate = 0.8552390552058609


In [40]:
#Gathers the accuracy, fp and tp rate of the autoencoder on a test set
colcount = len(xtest[0])
ae_out = autoencoder.predict([ xtest.reshape(-1, colcount, 1) ])
test = ae_out.reshape(2649,colcount)

ids = 0
acc = []
fplist = []
tplist = []

for things in test:
    guesses = []
    testset = []
    
    loops = 0
    for items in xtest[ids]:
        if items != 0:
            testset.append(loops)
        loops = loops + 1
    
    loops = 0
    for items in test[ids]:
        if items > 0.5:
            guesses.append(loops)
        loops = loops + 1
        
    correct = 0
    loops = 0
    tp = 0
    fp = 0
    for items in guesses:
        if items in testset:
            correct = correct + 1
            tp = tp + 1
        else:
            fp = fp + 1
        loops = loops + 1
    
    accuracy = correct / loops
    tprate = tp / len(guesses)
    fprate = fp / len(guesses)
    
    acc.append(accuracy)
    tplist.append(tprate)
    fplist.append(fprate)
    
    ids = ids + 1

totalacc = 0
totaltp = 0
totalfp = 0

loops = 0
for items in acc:
    totalacc = totalacc + items
    loops = loops + 1
netacc = totalacc/loops

loops = 0
for items in tplist:
    totaltp = totaltp + items
    loops = loops + 1
nettp = totaltp/loops

loops = 0
for items in fplist:
    totalfp = totalfp + items
    loops = loops + 1
netfp = totalfp/loops
    
netacc = totalacc/loops
nettp = totaltp/loops
netfp = totalfp/loops

#Calculates fn and tn rate

ids = 0
tnlist = []
fnlist = []

for things in test:
    guesses = []
    testset = []
    
    loops = 0
    for items in xtest[ids]:
        if items != 1:
            testset.append(loops)
        loops = loops + 1
    
    loops = 0
    for items in test[ids]:
        if items < 0.5:
            guesses.append(loops)
        loops = loops + 1
        
    correct = 0
    loops = 0
    tn = 0
    fn = 0
    for items in guesses:
        if items in testset:
            correct = correct + 1
            tn = tn + 1
        else:
            fn = fn + 1
        loops = loops + 1
    
    tnrate = tn / len(guesses)
    fnrate = fn / len(guesses)
    
    tnlist.append(tprate)
    fnlist.append(fprate)
    
    ids = ids + 1

totaltn = 0
totalfn = 0

loops = 0
for items in tnlist:
    totaltn = totaltn + items
    loops = loops + 1
nettn = totaltn/loops

loops = 0
for items in fnlist:
    totalfn = totalfn + items
    loops = loops + 1
netfn = totalfn/loops
    
nettn = totaltn/loops
netfn = totalfn/loops

print('fn rate =', netfn)
print('tn rate =', nettn)
print('net accuracy =', netacc)
print('fp rate =', netfp)
print('tp rate =', nettp)
print("epochs =", epochs)
print('batchsize =', batchsize)  
print('latent_space size =', latent_space)

83/83 [==============================] - 0s 511us/step


2024-05-20 21:24:58.315271: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [2649,334,1]
	 [[{{node Placeholder/_10}}]]
2024-05-20 21:24:58.315443: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [2649,334,1]
	 [[{{node Placeholder/_10}}]]


fn rate = 0.25
tn rate = 0.75
net accuracy = 0.8552390552058609
fp rate = 0.14476094479413892
tp rate = 0.8552390552058609
epochs = 4
batchsize = 3
latent_space size = 42


In [44]:
colcount = len(xtest[0])
ae_out = autoencoder.predict([ xtest.reshape(-1, colcount, 1) ])
predictions = ae_out.reshape(2649,colcount)

correctcount = 0
fpcount = 0
tpcount = 0
tncount = 0
fncount = 0

iterations = 0
for samples in predictions:
    loops = 0
    for value in predictions[iterations]:
        testscore = xtest[iterations][loops]
        if value > 0.5:
            if testscore != 0:
                correctcount = correctcount + 1
                tpcount = tpcount + 1
            else:
                fpcount = fpcount + 1
        else:
            if testscore != 0:
                fncount = fncount + 1
            else:
                correctcount = correctcount + 1
                tncount = tncount + 1
                
        loops = loops + 1

netfn = fncount / (fncount + tncount)
nettn = tncount / (fncount + tncount)
netacc = correctcount / (fpcount + fncount + tpcount + tncount)
netfp = fpcount / (fpcount + tpcount)
nettp = tpcount / (tpcount + fpcount)

print(correctcount)
print('fn rate =', netfn)
print('tn rate =', nettn)
print('net accuracy =', netacc)
print('fp rate =', netfp)
print('tp rate =', nettp)

83/83 [==============================] - 0s 519us/step


2024-05-20 21:28:51.483269: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [2649,334,1]
	 [[{{node Placeholder/_10}}]]
2024-05-20 21:28:51.483479: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [2649,334,1]
	 [[{{node Placeholder/_10}}]]


876819
884766
fn rate = 0.009036144578313253
tn rate = 0.9909638554216867
net accuracy = 0.9910179640718563
fp rate = 0.0
tp rate = 1.0


83/83 [==============================] - 0s 424us/step


2024-05-20 21:24:50.723100: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [2649,334,1]
	 [[{{node Placeholder/_10}}]]
2024-05-20 21:24:50.723215: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [2649,334,1]
	 [[{{node Placeholder/_10}}]]


fn rate = 0.009036144578313253
tn rate = 0.9909638554216867
net accuracy = 0.9910179640718563
fp rate = 0.0
tp rate = 1.0
